# Import Modules

In [29]:
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

# Prepare Data

In [32]:
source_x = pd.read_csv("./data/2_X_train.csv")
source_y = pd.read_csv("./data/2_Y_train.csv")
source_x_y = pd.concat([source_x, source_y], axis=1)
source_x_y["Domain"] = 1

target_x = pd.read_csv("./data/5_X_train.csv")
target_y = pd.read_csv("./data/5_Y_train.csv")
target_x_y = pd.concat([target_x, target_y], axis=1)
target_x_y["Domain"] = 0

num_for_domain = 900
data_for_domain = pd.concat([source_x_y, target_x_y[:num_for_domain]], axis=0)
data_for_task = target_x_y[num_for_domain:]
data_for_task = data_for_task.drop(columns="Domain")

In [31]:
source_x_y

,Ratio,Mean,Max,Min,Std,Range,Temp,Lag-1_Ratio,Lag+1_Ratio,Season,Sunday,Am,Lunch,Pm,Occupancy,Domain
0,0.291616,4.376024e+05,471791.255,403413.570,34188.8425,68377.685,15.0,0.506878,0.118813,1,1,1,0,0,1,1
1,0.118813,1.782926e+05,210867.162,145718.019,32574.5715,65149.143,16.5,0.291616,0.228141,1,1,1,0,0,1,1
2,0.228141,3.758871e+05,411026.725,340747.424,35139.6505,70279.301,15.5,0.118813,0.150397,1,1,1,0,0,1,1
3,0.150397,2.529675e+05,281994.830,223940.218,29027.3060,58054.612,15.5,0.228141,0.177673,1,1,1,0,0,1,1
4,0.177673,3.190740e+05,380877.247,257270.700,61803.2735,123606.547,16.0,0.150397,0.102678,1,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,0.384522,1.429647e+06,1921537.420,937757.055,491890.1825,983780.365,5.0,0.065421,0.217411,0,0,0,0,1,1,1
1356,0.217411,8.083327e+05,902196.241,714469.213,93863.5140,187727.028,5.0,0.384522,0.234310,0,0,0,0,1,1,1
1357,0.234310,7.319390e+05,749873.285,714004.804,17934.2405,35868.481,4.0,0.217411,0.462214,0,0,0,0,1,1,1
1358,0.462214,1.443867e+06,2196655.709,691079.162,752788.2735,1505576.547,4.0,0.234310,0.548960,0,0,0,0,1,1,1


In [22]:
domain_discriminator = LogisticRegression()
task_classifier = LogisticRegression()

# Domain Discrimination

In [23]:
x_for_domain = data_for_domain.drop(columns="Domain")
scaler = preprocessing.StandardScaler()
x_for_domain = scaler.fit_transform(x_for_domain)

y_for_domain = data_for_domain["Domain"]

In [24]:
domain_discriminator.fit(x_for_domain, y_for_domain.values)

LogisticRegression()

In [25]:
transferred_mask = domain_discriminator.predict(x_for_domain[:-num_for_domain]) == 0
num_transffered = sum(transferred_mask)
print(f"{num_transffered} samples transferred from Source to Target")

90 samples transferred from Source to Target


# Task Classification

In [26]:
transferred_data = pd.concat([data_for_domain[:-num_for_domain][transferred_mask], data_for_domain[-num_for_domain:]], axis=0) 
transferred_data = transferred_data.drop(columns=["Domain"])

transferred_x = transferred_data.drop(columns="Occupancy").values
transferred_x = scaler.fit_transform(transferred_x)
transferred_y = transferred_data["Occupancy"].values

task_x = data_for_task.drop(columns="Occupancy").values
task_y = data_for_task["Occupancy"].values

In [34]:
task_classifier.fit(transferred_x, transferred_y)
pred_y = task_classifier.predict(task_x)
acc = sum(pred_y == task_y)/task_y.shape[0]

print(f"+Adversarial Validation Accuracy:{acc}")

+Adversarial Validation Accuracy:0.8169014084507042


In [28]:
task_classifier.fit(transferred_x[num_transffered:], transferred_y[num_transffered:])
pred_y = task_classifier.predict(task_x)
acc = sum(pred_y == task_y)/task_y.shape[0]

print(f"-Adversarial Validation Accuracy:{acc}")

-Adversarial Validation Accuracy:0.8169014084507042
